In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import zscore
import statsmodels.formula.api as smf
import statistics
from scipy.stats import gaussian_kde
from scipy.integrate import quad
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Junior/Dyanne JP/ABCD_Release4.0_Tabular_Dataset.csv')
df_baseline = df[df['eventname'] == 'baseline_year_1_arm_1']
df_2year = df[df['eventname'] == '2_year_follow_up_y_arm_1']

In [ ]:
# All participants = 11879, baseline = 11876, 2 year = 10414
df['subjectkey'].nunique()
df_baseline['subjectkey'].nunique()
df_2year['subjectkey'].nunique()

In [ ]:
baseline_2year = pd.merge(df_baseline, df_2year, on='subjectkey', how='left', suffixes=('_baseline', '_2year'))

In [ ]:
print('Number of participants with baseline and 2-year data:')
both = baseline_2year.shape[0] - baseline_2year['eventname_2year'].isna().sum()
print(both)

In [ ]:
baseline_2year.dropna(subset=['eventname_2year'], inplace=True)

In [ ]:
print('Picture Vocabulary Test - Number of NaNs')
picvocab_bs_nan = baseline_2year['nihtbx_picvocab_uncorrected_baseline'].isna().sum()
picvocab_2_nan = baseline_2year['nihtbx_picvocab_uncorrected_2year'].isna().sum()
print(f'Baseline: {picvocab_bs_nan}')
print(f'2-year: {picvocab_2_nan}')
###
print('Flanker Inhibitory Control and Attention Test - Number of NaNs')
flanker_bs_nan = baseline_2year['nihtbx_flanker_uncorrected_baseline'].isna().sum()
flanker_2_nan = baseline_2year['nihtbx_flanker_uncorrected_2year'].isna().sum()
print(f'Baseline: {flanker_bs_nan}')
print(f'2-year: {flanker_2_nan}')
###
print('List Sorting Working Memory Test - Number of NaNs')
list_bs_nan = baseline_2year['nihtbx_list_uncorrected_baseline'].isna().sum()
list_2_nan = baseline_2year['nihtbx_list_uncorrected_2year'].isna().sum()
print(f'Baseline: {list_bs_nan}')
print(f'2-year: {list_2_nan}')
###
print('Dimensional Change Card Sort Test - Number of NaNs')
cardsort_bs_nan = baseline_2year['nihtbx_cardsort_uncorrected_baseline'].isna().sum()
cardsort_2_nan = baseline_2year['nihtbx_cardsort_uncorrected_2year'].isna().sum()
print(f'Baseline: {cardsort_bs_nan}')
print(f'2-year: {cardsort_2_nan}')
###
print('Pattern Comparison Processing Speed Test - Number of NaNs')
pattern_bs_nan = baseline_2year['nihtbx_pattern_uncorrected_baseline'].isna().sum()
pattern_2_nan = baseline_2year['nihtbx_pattern_uncorrected_2year'].isna().sum()
print(f'Baseline: {pattern_bs_nan}')
print(f'2-year: {pattern_2_nan}')
###
print('Picture Sequence Memory Test - Number of NaNs')
picture_bs_nan = baseline_2year['nihtbx_picture_uncorrected_baseline'].isna().sum()
picture_2_nan = baseline_2year['nihtbx_picture_uncorrected_2year'].isna().sum()
print(f'Baseline: {picture_bs_nan}')
print(f'2-year: {picture_2_nan}')
###
print('Oral Reading Recognition Test - Number of NaNs')
reading_bs_nan = baseline_2year['nihtbx_reading_uncorrected_baseline'].isna().sum()
reading_2_nan = baseline_2year['nihtbx_reading_uncorrected_2year'].isna().sum()
print(f'Baseline: {reading_bs_nan}')
print(f'2-year: {reading_2_nan}')

In [ ]:
check = ['nihtbx_picvocab_uncorrected_baseline','nihtbx_picvocab_uncorrected_2year','nihtbx_flanker_uncorrected_baseline',
         'nihtbx_flanker_uncorrected_2year','nihtbx_pattern_uncorrected_baseline','nihtbx_pattern_uncorrected_2year',
         'nihtbx_picture_uncorrected_baseline','nihtbx_picture_uncorrected_2year','nihtbx_reading_uncorrected_baseline',
         'nihtbx_reading_uncorrected_2year','nihtbx_cryst_uncorrected_baseline','nihtbx_cryst_uncorrected_2year']

cleaned = baseline_2year.dropna(subset=check)
#(7172,1055)

# drop columns that only have NaN values
cleaned.dropna(axis=1, how='all', inplace=True)
#(7172,892)

print('Number of participants with all 5 test scores')
print(cleaned.shape[0])

In [ ]:
non_numeric_columns = cleaned.select_dtypes(exclude=['number']).columns

print("Non-numeric columns:", non_numeric_columns.tolist())

In [ ]:
(cleaned['sex_baseline']==cleaned['sex_2year']).sum()

In [ ]:
cleaned = cleaned.drop('sex_2year',axis=1)

In [ ]:
cleaned['sex_baseline'] = cleaned['sex_baseline'].replace({'M': 0, 'F': 1})

In [ ]:
cleaned['income_baseline'].unique()

In [ ]:
from sklearn.impute import SimpleImputer

def medianimpute(df):
    # Create a copy of the DataFrame to avoid changing the original data
    df_imputed = df.copy()

    # Identify numeric columns by data type
    numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns

    # Define the imputer with a median strategy
    imputer = SimpleImputer(strategy='median')

    # Apply the imputer only to the numeric columns
    df_imputed[numeric_cols] = imputer.fit_transform(df_imputed[numeric_cols])

    return df_imputed

In [ ]:
imputed = medianimpute(cleaned)

In [ ]:
print('Number of families in the sample')
imputed['family_id_baseline'].nunique()

In [ ]:
# nest family in ABCD study site for LME random effect
imputed['site_family'] = imputed['abcd_site_baseline'].astype(str) + "_" + imputed['family_id_baseline'].astype(str)
imputed['site_family'] = imputed['abcd_site_2year'].astype(str) + "_" + imputed['family_id_baseline'].astype(str)

In [ ]:
imputed['income_baseline'].unique()

In [ ]:
df_low = imputed[(imputed['income_baseline'] >= 1) & (imputed['income_baseline'] < 7)]
df_med = imputed[imputed['income_baseline'].isin([7])]
df_high = imputed[(imputed['income_baseline'] > 7) & (imputed['income_baseline'] <= 10)]

In [ ]:
print(df_low.shape[0]) # 1785 in low SES group
print(df_med.shape[0]) # 931 in median SES group (7)
print(df_high.shape[0]) # 4456 in high SES group

In [ ]:
#plt.style.use('default')

plt.hist(imputed['income_baseline'],alpha=0.6)
plt.xlabel('Income',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.xlim(1,10)
plt.axvline(x=7, color='r', linestyle='--')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.text(2, 1800, 'Low SES: n=1,785\nHigh SES: n=4,456', fontsize=15, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show()

In [ ]:
# stacked bar chart
plt.hist([df_high['race_ethnicity_baseline'], df_low['race_ethnicity_baseline']], bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5], rwidth=0.9, alpha=0.6,
         stacked=True, label=['High SES', 'Low SES'])
# plt.hist([df_high['race_ethnicity_baseline'], df_low['race_ethnicity_baseline']], bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5], rwidth=0.9, alpha=0.6,
#          stacked=True, label=['High SES', 'Low SES'],density=True)
plt.xlim(0.5, 5.5)
labels = ['White', 'Black', 'Hispanic', 'Asian', 'Other']
plt.xticks(range(1, 6),labels,fontsize=15)
plt.xlabel('Race',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
# plt.ylabel('Density',fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_picvocab_uncorrected_baseline'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 38,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Picture Vocabulary Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('Baseline', fontsize=13)
plt.ylim(35,130)
plt.show()

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_picvocab_uncorrected_2year'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 38,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Picture Vocabulary Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('2-year Follow-up', fontsize=13)
plt.ylim(35,130)
plt.show()

In [ ]:
# # Set y lim
# print(imputed['nihtbx_flanker_uncorrected_baseline'].min())
# print(imputed['nihtbx_flanker_uncorrected_2year'].min())
# print(imputed['nihtbx_flanker_uncorrected_baseline'].max())
# print(imputed['nihtbx_flanker_uncorrected_2year'].max())

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_flanker_uncorrected_baseline'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 52,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Flanker Inhibitory Control and Attention Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('Baseline', fontsize=13)
plt.ylim(50,120)
plt.show()

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_flanker_uncorrected_2year'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 52,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Flanker Inhibitory Control and Attention Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('2-year Follow-up', fontsize=13)
plt.ylim(50,120)
plt.show()

In [ ]:
# # Set y lim
# print(imputed['nihtbx_pattern_uncorrected_baseline'].min())
# print(imputed['nihtbx_pattern_uncorrected_2year'].min())
# print(imputed['nihtbx_pattern_uncorrected_baseline'].max())
# print(imputed['nihtbx_pattern_uncorrected_2year'].max())

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_pattern_uncorrected_baseline'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 29,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Pattern Comparison Processing Speed Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('Baseline', fontsize=13)
plt.ylim(25,170)
plt.show()

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_pattern_uncorrected_2year'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 29,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Pattern Comparison Processing Speed Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('2-year Follow-up', fontsize=13)
plt.ylim(25,170)
plt.show()

In [ ]:
# # Set ylim
# print(imputed['nihtbx_picture_uncorrected_baseline'].min())
# print(imputed['nihtbx_picture_uncorrected_2year'].min())
# print(imputed['nihtbx_picture_uncorrected_baseline'].max())
# print(imputed['nihtbx_picture_uncorrected_2year'].max())

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_picture_uncorrected_baseline'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 71.5,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Picture Sequence Memory Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('Baseline', fontsize=13)
plt.ylim(70,140)
plt.show()

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_picture_uncorrected_2year'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 71.5,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Picture Sequence Memory Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('2-year Follow-up', fontsize=13)
plt.ylim(70,140)
plt.show()

In [ ]:
# # Set y lim
# print(imputed['nihtbx_reading_uncorrected_baseline'].min())
# print(imputed['nihtbx_reading_uncorrected_2year'].min())
# print(imputed['nihtbx_reading_uncorrected_baseline'].max())
# print(imputed['nihtbx_reading_uncorrected_2year'].max())

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_reading_uncorrected_baseline'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 60,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('Baseline', fontsize=13)
plt.ylim(55,125)
plt.show()

In [ ]:
x = imputed['income_baseline'].values
y = imputed['nihtbx_reading_uncorrected_2year'].values

# Add constant to x to represent the intercept
x = sm.add_constant(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12)

# Fit the model using statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary to see the p-value, coefficients, R-squared, etc.
print(model.summary())

# Making predictions on the test set
y_pred = model.predict(x_test)

# Calculate R^2 score for the test set
r2_test = metrics.r2_score(y_test, y_pred)
print(f"R^2 score for the test set: {r2_test:.4f}")

p_value = model.pvalues[1]  # Assuming index 1 is for 'income_baseline'
print(f"The p-value for the income coefficient is: {p_value}")

b, w = model.params

plt.scatter(x_test[:, 1], y_test, color='blue', alpha=0.5)  # x_test[:, 1] because x_test includes the constant
plt.plot(x_test[:, 1], y_pred, color='red')
plt.text(1, 60,f"y={w:0.2f}x+{b:0.2f}", fontsize=13)
plt.xticks(ticks=[1,2,3,4,5,6,7,8,9,10], labels=['1','2','3','4','5','6','7','8','9','10'], fontsize=13)
plt.xlabel('Income',fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=12)
plt.yticks(fontsize=13)
plt.title('2-year Follow-up', fontsize=13)
plt.ylim(55,125)
plt.show()

In [ ]:
high_diff_picvocab = df_high['nihtbx_picvocab_uncorrected_2year']-df_high['nihtbx_picvocab_uncorrected_baseline']
low_diff_picvocab = df_low['nihtbx_picvocab_uncorrected_2year']-df_low['nihtbx_picvocab_uncorrected_baseline']

high_diff_flanker = df_high['nihtbx_flanker_uncorrected_2year']-df_high['nihtbx_flanker_uncorrected_baseline']
low_diff_flanker = df_low['nihtbx_flanker_uncorrected_2year']-df_low['nihtbx_flanker_uncorrected_baseline']

high_diff_pattern = df_high['nihtbx_pattern_uncorrected_2year']-df_high['nihtbx_pattern_uncorrected_baseline']
low_diff_pattern = df_low['nihtbx_pattern_uncorrected_2year']-df_low['nihtbx_pattern_uncorrected_baseline']

high_diff_picture = df_high['nihtbx_picture_uncorrected_2year']-df_high['nihtbx_picture_uncorrected_baseline']
low_diff_picture = df_low['nihtbx_picture_uncorrected_2year']-df_low['nihtbx_picture_uncorrected_baseline']

high_diff_reading = df_high['nihtbx_reading_uncorrected_2year']-df_high['nihtbx_reading_uncorrected_baseline']
low_diff_reading = df_low['nihtbx_reading_uncorrected_2year']-df_low['nihtbx_reading_uncorrected_baseline']

In [ ]:
high_prop_picvocab_pos = (high_diff_picvocab > 0).sum() / high_diff_picvocab.size
low_prop_picvocab_pos = (low_diff_picvocab > 0).sum() / low_diff_picvocab.size
high_prop_picvocab_neg = (high_diff_picvocab < 0).sum() / high_diff_picvocab.size
low_prop_picvocab_neg = (low_diff_picvocab < 0).sum() / low_diff_picvocab.size

high_prop_flanker_pos = (high_diff_flanker > 0).sum() / high_diff_flanker.size
low_prop_flanker_pos = (low_diff_flanker > 0).sum() / low_diff_flanker.size
high_prop_flanker_neg = (high_diff_flanker < 0).sum() / high_diff_flanker.size
low_prop_flanker_neg = (low_diff_flanker < 0).sum() / low_diff_flanker.size

high_prop_pattern_pos = (high_diff_pattern > 0).sum() / high_diff_pattern.size
low_prop_pattern_pos = (low_diff_pattern > 0).sum() / low_diff_pattern.size
high_prop_pattern_neg = (high_diff_pattern < 0).sum() / high_diff_pattern.size
low_prop_pattern_neg = (low_diff_pattern < 0).sum() / low_diff_pattern.size

high_prop_picture_pos = (high_diff_picture > 0).sum() / high_diff_picture.size
low_prop_picture_pos = (low_diff_picture > 0).sum() / low_diff_picture.size
high_prop_picture_neg = (high_diff_picture < 0).sum() / high_diff_picture.size
low_prop_picture_neg = (low_diff_picture < 0).sum() / low_diff_picture.size

high_prop_reading_pos = (high_diff_reading > 0).sum() / high_diff_reading.size
low_prop_reading_pos = (low_diff_reading > 0).sum() / low_diff_reading.size
high_prop_reading_neg = (high_diff_reading < 0).sum() / high_diff_reading.size
low_prop_reading_neg = (low_diff_reading < 0).sum() / low_diff_reading.size

In [ ]:
high_kde_picvocab = gaussian_kde(high_diff_picvocab)
low_kde_picvocab = gaussian_kde(low_diff_picvocab)

high_area_picvocab_pos, _ = quad(high_kde_picvocab, 0, np.inf)
low_area_picvocab_pos, _ = quad(low_kde_picvocab, 0, np.inf)
high_area_picvocab_neg, _ = quad(high_kde_picvocab, -np.inf, 0)
low_area_picvocab_neg, _ = quad(low_kde_picvocab, -np.inf, 0)
##
high_kde_flanker = gaussian_kde(high_diff_flanker)
low_kde_flanker = gaussian_kde(low_diff_flanker)

high_area_flanker_pos, _ = quad(high_kde_flanker, 0, np.inf)
low_area_flanker_pos, _ = quad(low_kde_flanker, 0, np.inf)
high_area_flanker_neg, _ = quad(high_kde_flanker, -np.inf, 0)
low_area_flanker_neg, _ = quad(low_kde_flanker, -np.inf, 0)
##
high_kde_pattern = gaussian_kde(high_diff_pattern)
low_kde_pattern = gaussian_kde(low_diff_pattern)

high_area_pattern_pos, _ = quad(high_kde_pattern, 0, np.inf)
low_area_pattern_pos, _ = quad(low_kde_pattern, 0, np.inf)
high_area_pattern_neg, _ = quad(high_kde_pattern, -np.inf, 0)
low_area_pattern_neg, _ = quad(low_kde_pattern, -np.inf, 0)
##
high_kde_picture = gaussian_kde(high_diff_picture)
low_kde_picture = gaussian_kde(low_diff_picture)

high_area_picture_pos, _ = quad(high_kde_picture, 0, np.inf)
low_area_picture_pos, _ = quad(low_kde_picture, 0, np.inf)
high_area_picture_neg, _ = quad(high_kde_picture, -np.inf, 0)
low_area_picture_neg, _ = quad(low_kde_picture, -np.inf, 0)
##
high_kde_reading = gaussian_kde(high_diff_reading)
low_kde_reading = gaussian_kde(low_diff_reading)

high_area_reading_pos, _ = quad(high_kde_reading, 0, np.inf)
low_area_reading_pos, _ = quad(low_kde_reading, 0, np.inf)
high_area_reading_neg, _ = quad(high_kde_reading, -np.inf, 0)
low_area_reading_neg, _ = quad(low_kde_reading, -np.inf, 0)

In [ ]:
# Calculate the proportions for the High SES group
counts_high, _ = np.histogram(high_diff_picvocab, bins=25)
weights_high = np.ones_like(high_diff_picvocab) / len(high_diff_picvocab)

# Calculate the proportions for the Low SES group
counts_low, _ = np.histogram(low_diff_picvocab, bins=25)
weights_low = np.ones_like(low_diff_picvocab) / len(low_diff_picvocab)

data_combined = np.concatenate((high_diff_picvocab, low_diff_picvocab))
bins = np.histogram_bin_edges(data_combined, bins=25)

# Plot the histograms with proportions on the y-axis
plt.hist(high_diff_picvocab, bins=bins, alpha=0.4, label='High SES', weights=weights_high)
plt.hist(low_diff_picvocab, bins=bins, alpha=0.4, label='Low SES', weights=weights_low)

# Plot customization
plt.xlabel('Picture Vocabulary Test (2-year - baseline)', fontsize=13)
plt.ylabel('Proportion', fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(25, 0.1, f'High SES > 0: {high_prop_picvocab_pos:0.2f}\nLow SES > 0: {low_prop_picvocab_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-39, 0.1, f'High SES < 0: {high_prop_picvocab_neg:0.2f}\nLow SES < 0: {low_prop_picvocab_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show()

In [ ]:
# kernel density estimation
sns.kdeplot(high_diff_picvocab, color='blue', label='High SES')
sns.kdeplot(low_diff_picvocab, color='orange', label='Low SES')
plt.xlabel('Picture Vocabulary Test (2-year - baseline)',fontsize=13)
plt.ylabel('Density',fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(25, 0.03, f'High SES > 0: {high_area_picvocab_pos:0.2f}\nLow SES > 0: {low_area_picvocab_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-43, 0.03, f'High SES < 0: {high_area_picvocab_neg:0.2f}\nLow SES < 0: {low_area_picvocab_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show();

In [ ]:
# Calculate the proportions for the High SES group
counts_high, _ = np.histogram(high_diff_flanker, bins=25)
weights_high = np.ones_like(high_diff_flanker) / len(high_diff_flanker)

# Calculate the proportions for the Low SES group
counts_low, _ = np.histogram(low_diff_flanker, bins=25)
weights_low = np.ones_like(low_diff_flanker) / len(low_diff_flanker)

data_combined = np.concatenate((high_diff_flanker, low_diff_flanker))
bins = np.histogram_bin_edges(data_combined, bins=25)

# Plot the histograms with proportions on the y-axis
plt.hist(high_diff_flanker, bins=bins, alpha=0.4, label='High SES', weights=weights_high)
plt.hist(low_diff_flanker, bins=bins, alpha=0.4, label='Low SES', weights=weights_low)

# Plot customization
plt.xlabel('Flanker Inhibitory Control and Attention Test (2-year - baseline)', fontsize=11.5)
plt.ylabel('Proportion', fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.ylim(0,0.25)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(23, 0.1, f'High SES > 0: {high_prop_flanker_pos:0.2f}\nLow SES > 0: {low_prop_flanker_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-35, 0.1, f'High SES < 0: {high_prop_flanker_neg:0.2f}\nLow SES < 0: {low_prop_flanker_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show()

In [ ]:
# kernel density estimation
sns.kdeplot(high_diff_flanker, color='blue', label='High SES')
sns.kdeplot(low_diff_flanker, color='orange', label='Low SES')
plt.xlabel('Flanker Inhibitory Control and Attention Test (2-year - baseline)',fontsize=11.5)
plt.ylabel('Density',fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(25, 0.03, f'High SES > 0: {high_area_flanker_pos:0.2f}\nLow SES > 0: {low_area_flanker_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-42, 0.03, f'High SES < 0: {high_area_flanker_neg:0.2f}\nLow SES < 0: {low_area_flanker_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show();

In [ ]:
# Calculate the proportions for the High SES group
counts_high, _ = np.histogram(high_diff_pattern, bins=25)
weights_high = np.ones_like(high_diff_pattern) / len(high_diff_pattern)

# Calculate the proportions for the Low SES group
counts_low, _ = np.histogram(low_diff_pattern, bins=25)
weights_low = np.ones_like(low_diff_pattern) / len(low_diff_pattern)

data_combined = np.concatenate((high_diff_pattern, low_diff_pattern))
bins = np.histogram_bin_edges(data_combined, bins=25)

# Plot the histograms with proportions on the y-axis
plt.hist(high_diff_pattern, bins=bins, alpha=0.4, label='High SES', weights=weights_high)
plt.hist(low_diff_pattern, bins=bins, alpha=0.4, label='Low SES', weights=weights_low)

# Plot customization
plt.xlabel('Pattern Comparison Processing Speed Test (2-year - baseline)', fontsize=11.5)
plt.ylabel('Proportion', fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.ylim(0,0.25)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(40, 0.1, f'High SES > 0: {high_prop_pattern_pos:0.2f}\nLow SES > 0: {low_prop_pattern_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-55, 0.1, f'High SES < 0: {high_prop_pattern_neg:0.2f}\nLow SES < 0: {low_prop_pattern_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show()

In [ ]:
# kernel density estimation
sns.kdeplot(high_diff_pattern, color='blue', label='High SES')
sns.kdeplot(low_diff_pattern, color='orange', label='Low SES')
plt.xlabel('Pattern Comparison Processing Speed Test (2-year - baseline)',fontsize=11.5)
plt.ylabel('Density',fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(45, 0.01, f'High SES > 0: {high_area_pattern_pos:0.2f}\nLow SES > 0: {low_area_pattern_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-65,0.01, f'High SES < 0: {high_area_pattern_neg:0.2f}\nLow SES < 0: {low_area_pattern_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show();

In [ ]:
# Calculate the proportions for the High SES group
counts_high, _ = np.histogram(high_diff_picture, bins=25)
weights_high = np.ones_like(high_diff_picture) / len(high_diff_picture)

# Calculate the proportions for the Low SES group
counts_low, _ = np.histogram(low_diff_picture, bins=25)
weights_low = np.ones_like(low_diff_picture) / len(low_diff_picture)

data_combined = np.concatenate((high_diff_picture, low_diff_picture))
bins = np.histogram_bin_edges(data_combined, bins=25)

# Plot the histograms with proportions on the y-axis
plt.hist(high_diff_picture, bins=bins, alpha=0.4, label='High SES', weights=weights_high)
plt.hist(low_diff_picture, bins=bins, alpha=0.4, label='Low SES', weights=weights_low)

# Plot customization
plt.xlabel('Picture Sequence Memory Test (2-year - baseline)', fontsize=11.5)
plt.ylabel('Proportion', fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.ylim(0,0.25)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(23, 0.1, f'High SES > 0: {high_prop_picture_pos:0.2f}\nLow SES > 0: {low_prop_picture_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-46, 0.1, f'High SES < 0: {high_prop_picture_neg:0.2f}\nLow SES < 0: {low_prop_picture_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show()

In [ ]:
# kernel density estimation
sns.kdeplot(high_diff_picture, color='blue', label='High SES')
sns.kdeplot(low_diff_picture, color='orange', label='Low SES')
plt.xlabel('Picture Sequence Memory Test (2-year - baseline)',fontsize=11.5)
plt.ylabel('Density',fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(28, 0.01, f'High SES > 0: {high_area_picture_pos:0.2f}\nLow SES > 0: {low_area_picture_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-55,0.01, f'High SES < 0: {high_area_picture_neg:0.2f}\nLow SES < 0: {low_area_picture_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show();

In [ ]:
# Calculate the proportions for the High SES group
counts_high, _ = np.histogram(high_diff_reading, bins=25)
weights_high = np.ones_like(high_diff_reading) / len(high_diff_reading)

# Calculate the proportions for the Low SES group
counts_low, _ = np.histogram(low_diff_reading, bins=25)
weights_low = np.ones_like(low_diff_reading) / len(low_diff_reading)

data_combined = np.concatenate((high_diff_reading, low_diff_reading))
bins = np.histogram_bin_edges(data_combined, bins=25)

# Plot the histograms with proportions on the y-axis
plt.hist(high_diff_reading, bins=bins, alpha=0.4, label='High SES', weights=weights_high)
plt.hist(low_diff_reading, bins=bins, alpha=0.4, label='Low SES', weights=weights_low)

# Plot customization
plt.xlabel('Oral Reading Recognition Test (2-year - baseline)', fontsize=11.5)
plt.ylabel('Proportion', fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.ylim(0,0.55)
plt.xlim(-35,65)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(33, 0.18, f'High SES > 0: {high_prop_reading_pos:0.2f}\nLow SES > 0: {low_prop_reading_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-32, 0.18, f'High SES < 0: {high_prop_reading_neg:0.2f}\nLow SES < 0: {low_prop_reading_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show()

In [ ]:
# kernel density estimation
sns.kdeplot(high_diff_reading, color='blue', label='High SES')
sns.kdeplot(low_diff_reading, color='orange', label='Low SES')
plt.xlabel('Oral Reading Recognition Test (2-year - baseline)',fontsize=11.5)
plt.ylabel('Density',fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.xlim(-35,65)
plt.axvline(x=0, color='r', linestyle='--')
plt.legend(fontsize=13)
plt.text(33, 0.03, f'High SES > 0: {high_area_reading_pos:0.2f}\nLow SES > 0: {low_area_reading_pos:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.text(-33,0.03, f'High SES < 0: {high_area_reading_neg:0.2f}\nLow SES < 0: {low_area_reading_neg:0.2f}', fontsize=11, color='black',
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
plt.show();

In [ ]:
def moving_average_with_overlaps(data_x, data_y, window_size, overlap):
    range_x = np.max(data_x)-np.min(data_x)
    step_size = int(window_size * (1 - overlap))
    num_windows = int((range_x - window_size) // step_size + 1)

    moving_avg = np.zeros(num_windows)

    x0 = np.min(data_x)
    for i in range(len(moving_avg)):
        x1 = x0 + window_size
        filt_0 = data_x > x0
        filt_1 = data_x < x1
        filt = filt_0*filt_1
        moving_avg[i] = np.nanmean(data_y[filt])
        x0 += step_size

    xaxis = np.arange(np.min(data_x),np.max(data_x),range_x/len(moving_avg))

    return xaxis,moving_avg

window_size = 9
overlap_ratio = 0.75

In [ ]:
low_baseline_age = np.array(df_low.loc[df_low['eventname_baseline'] == 'baseline_year_1_arm_1', 'age_baseline'])
low_2year_age = np.array(df_low.loc[df_low['eventname_2year'] == '2_year_follow_up_y_arm_1', 'age_2year'])

high_baseline_age = np.array(df_high.loc[df_high['eventname_baseline'] == 'baseline_year_1_arm_1', 'age_baseline'])
high_2year_age = np.array(df_high.loc[df_high['eventname_2year'] == '2_year_follow_up_y_arm_1', 'age_2year'])

In [ ]:
low_baseline_picvocab = np.array(df_low.loc[df_low['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_picvocab_uncorrected_baseline'])
low_2year_picvocab = np.array(df_low.loc[df_low['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_picvocab_uncorrected_2year'])
high_baseline_picvocab = np.array(df_high.loc[df_high['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_picvocab_uncorrected_baseline'])
high_2year_picvocab = np.array(df_high.loc[df_high['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_picvocab_uncorrected_2year'])

low_baseline_flanker = np.array(df_low.loc[df_low['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_flanker_uncorrected_baseline'])
low_2year_flanker = np.array(df_low.loc[df_low['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_flanker_uncorrected_2year'])
high_baseline_flanker = np.array(df_high.loc[df_high['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_flanker_uncorrected_baseline'])
high_2year_flanker = np.array(df_high.loc[df_high['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_flanker_uncorrected_2year'])

low_baseline_pattern = np.array(df_low.loc[df_low['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_pattern_uncorrected_baseline'])
low_2year_pattern = np.array(df_low.loc[df_low['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_pattern_uncorrected_2year'])
high_baseline_pattern = np.array(df_high.loc[df_high['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_pattern_uncorrected_baseline'])
high_2year_pattern = np.array(df_high.loc[df_high['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_pattern_uncorrected_2year'])

low_baseline_picture = np.array(df_low.loc[df_low['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_picture_uncorrected_baseline'])
low_2year_picture = np.array(df_low.loc[df_low['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_picture_uncorrected_2year'])
high_baseline_picture = np.array(df_high.loc[df_high['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_picture_uncorrected_baseline'])
high_2year_picture = np.array(df_high.loc[df_high['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_picture_uncorrected_2year'])

low_baseline_reading = np.array(df_low.loc[df_low['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_reading_uncorrected_baseline'])
low_2year_reading = np.array(df_low.loc[df_low['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_reading_uncorrected_2year'])
high_baseline_reading = np.array(df_high.loc[df_high['eventname_baseline'] == 'baseline_year_1_arm_1', 'nihtbx_reading_uncorrected_baseline'])
high_2year_reading = np.array(df_high.loc[df_high['eventname_2year'] == '2_year_follow_up_y_arm_1', 'nihtbx_reading_uncorrected_2year'])

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(low_baseline_age,low_baseline_picvocab, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(low_2year_age,low_2year_picvocab, window_size, overlap_ratio)

In [ ]:
plt.plot(low_baseline_age, low_baseline_picvocab,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(low_2year_age, low_2year_picvocab,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.ylim(34,125)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Vocabulary Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.ylim(78,95)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Vocabulary Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_picvocab, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age,high_2year_picvocab, window_size, overlap_ratio)

In [ ]:
plt.plot(high_baseline_age, high_baseline_picvocab,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(high_2year_age, high_2year_picvocab,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.ylim(34,125)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Vocabulary Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.ylim(78,95)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Vocabulary Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(low_baseline_age,low_baseline_flanker, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(low_2year_age,low_2year_flanker, window_size, overlap_ratio)

In [ ]:
plt.plot(low_baseline_age, low_baseline_flanker,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(low_2year_age, low_2year_flanker,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Flanker Inhibitory Control and Attention Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.ylim(90,103)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Flanker Inhibitory Control and Attention Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_flanker, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age,high_2year_flanker, window_size, overlap_ratio)

In [ ]:
plt.plot(high_baseline_age, high_baseline_flanker,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(high_2year_age, high_2year_flanker,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Flanker Inhibitory Control and Attention Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.ylim(90,103)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Flanker Inhibitory Control and Attention Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(low_baseline_age,low_baseline_pattern, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(low_2year_age,low_2year_pattern, window_size, overlap_ratio)

In [ ]:
plt.plot(low_baseline_age, low_baseline_pattern,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(low_2year_age, low_2year_pattern,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Pattern Comparison Processing Speed Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.ylim(80,112)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Pattern Comparison Processing Speed Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_pattern, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age,high_2year_pattern, window_size, overlap_ratio)

In [ ]:
plt.plot(high_baseline_age, high_baseline_pattern,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(high_2year_age, high_2year_pattern,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Pattern Comparison Processing Speed Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.ylim(80,112)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Pattern Comparison Processing Speed Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(low_baseline_age,low_baseline_picture, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(low_2year_age,low_2year_picture, window_size, overlap_ratio)

In [ ]:
plt.plot(low_baseline_age, low_baseline_picture,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(low_2year_age, low_2year_picture,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Sequence Memory Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.ylim(98,114)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Sequence Memory Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_picture, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age,high_2year_picture, window_size, overlap_ratio)

In [ ]:
plt.plot(high_baseline_age, high_baseline_picture,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(high_2year_age, high_2year_picture,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Sequence Memory Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.ylim(98,114)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Picture Sequence Memory Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(low_baseline_age,low_baseline_reading, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(low_2year_age,low_2year_reading, window_size, overlap_ratio)

In [ ]:
plt.plot(low_baseline_age, low_baseline_reading,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(low_2year_age, low_2year_reading,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,164)
plt.ylim(86,98)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Low SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_reading, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age,high_2year_reading, window_size, overlap_ratio)

In [ ]:
plt.plot(high_baseline_age, high_baseline_reading,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(high_2year_age, high_2year_reading,'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
# exclude outlier

outlier = high_2year_reading.max()
# print(outlier)

filtered_high_2year_reading = high_2year_reading < outlier

xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_reading, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age[filtered_high_2year_reading],high_2year_reading[filtered_high_2year_reading], window_size, overlap_ratio)

plt.plot(high_baseline_age, high_baseline_reading,'b.',alpha=0.15)
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(high_2year_age[filtered_high_2year_reading], high_2year_reading[filtered_high_2year_reading],'r.',alpha=0.15)
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()

In [ ]:
# exclude outlier

outlier = high_2year_reading.max()
# print(outlier)

filtered_high_2year_reading = high_2year_reading < outlier

xs_bs, moving_avg_cog_bs = moving_average_with_overlaps(high_baseline_age,high_baseline_reading, window_size, overlap_ratio)
xs_fu, moving_avg_cog_fu = moving_average_with_overlaps(high_2year_age[filtered_high_2year_reading],high_2year_reading[filtered_high_2year_reading], window_size, overlap_ratio)

plt.plot(xs_bs, moving_avg_cog_bs,'b',linewidth=3,label='Baseline')
plt.plot(xs_fu, moving_avg_cog_fu,'r',linewidth=3,label='2 year')
plt.xlabel('Age (years)',fontsize=13)
plt.xlim(106,167)
plt.ylim(86,98)
plt.xticks(ticks=[108, 120, 132, 144, 156], labels=['9', '10', '11', '12', '13'], fontsize=13)
plt.ylabel('Oral Reading Recognition Test',fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('High SES',fontsize=15)
plt.legend(loc='lower right')
plt.show()